# Facility Location with patterns

In this task you need to solve the Facility Location Problem using patterns. 
You are given the following data:

* a range $I$ containing the potential locations where a facility can be opened
* a range $J$ containing the customers
* a tuple $b$ containing the customer demands $b_{j} \in \mathbb{Z}_{+}$ for every $j \in J$
* a tuple $f$ containing the opening costs $f_{i} \in \mathbb{Z}_{+}$ for each location $i \in I$
* a tuple $C$ containing the capacities $C_{i} \in \mathbb{Z}_{+}$ for each location $i \in I$
* a dictionary $c$ containing the connection costs $c_{i, j}$ for connecting facitily $i \in I$ with customer $j \in J$
* a tuple $P$ containing the power set of $J$

Your task is to find a feasible assignment of customers to facilities of minimum total cost, such that:

* every customer is assigned to exactly one opened facility
* total demand of all customers assigned to a facility $i \in I$ is at most its capacity $C_{i}$

The binary variable $l_{i, p}$ equals to $1$ if the subset of customers $p \subseteq J$ is assigned to the facility $i \in I$, otherwise $l_{i, p}$ equals to $0$.

For facility $i \in I$ let $\mathscr{P}^{i}$ the set of all "patterns", i.e. the power set of $J$
We define the costs $c_{P}^{i}$ of pattern $P \in \mathscr{P}^{i}$ for facility $i \in I$ by
$$c_{P}^{i} := \left\{\begin{array}{ll} f_{i} + \sum_{j \in P}{c_{ij}}, & \text{if } P \not = \emptyset \\
         0, & \text{if } P = \emptyset \end{array}\right. .$$


In [1]:
from gurobipy import *


def solve(I, J, b, f, C, c, P):
  model = Model('FacilityLocation')

  l = {}
  for i in I:
    for p in P:
      l[i, p] = model.addVar(vtype=GRB.BINARY, name=f'l[{i},{p}]')

  cost = {}
  for i in I:
    for p in P:
      if len(p) == 0:
        cost[i, p] = 0
      else:
        cost[i, p] = f[i] + quicksum(c[i, j] for j in p)

  # set objective function
  model.setObjective(quicksum(cost[i, p] * l[i, p] for i in I for p in P), GRB.MINIMIZE)

  # add constraints 

  # for facility i in I we exclude the non-feasible pattern, i.e. subsets of customers that cannot be assigned to it
  # at the same time
  for i in I:
    for p in P:
      if sum(b[j] for j in list(p)) > C[i]:
        model.addConstr(l[i, p] == 0)

  # for each facility i in I, choose exactly ONE pattern
  for i in I:
    model.addConstr(quicksum(l[i, p] for p in P) == 1)

  # for each customer j in J, choose exactly ONE pattern that asssigns them to a facility
  for j in J:
    model.addConstr(quicksum(l[i, p] for i in I for p in P if j in list(p)) == 1)

  model.update()
  model.optimize()
  
  return model